In [1]:
import numpy as np
import matplotlib.pyplot as plt

Посчитаем моменты в общем случае
$$
\int_{\gamma_1}^{\gamma2} p(x) f(x) d x 
$$
Введем замену переменной $x = b - t$.
Тогда задача сведется к следующей.
$$
\int_{\gamma_2 - b}^{\gamma_1 - b} p(b - t) f(b - t) d t = \int_{\gamma_2 - b}^{\gamma_1 - b} \tilde p(t) f(b - t) d t \approx \sum_{i=1}^{n} A_{i} f\left(b - t_{i}\right)
$$
где $\tilde p(t) =  t^{-\frac{1}{4}}$. Тогда
$$\mu_i = \int_{\gamma_2 - b}^{\gamma_1 - b} t^{-\frac{1}{4}} \cdot t^{i} dt = \frac{1}{i+1-\frac{1}{4}}x^{i+1-\frac{1}{4}} \bigg|^{b - \gamma_1}_{b - \gamma_2}$$


In [2]:
def moment(gamma1, gamma2, k):
    """ Функция вычисления моментов """
    b = 3.2
    coef = k + 1 - 1/4
    return 1 / coef * ((b - gamma1) ** coef - (b - gamma2) ** coef)

In [3]:
def f(x, a=1.7, b=3.2, alpha=0, beta=0.25):
    return 3 * np.cos(2 * x) * np.exp(2 * x / 3) + 5 * np.sin(2.5 * x) * np.exp(-x / 3) + 2 * x

In [13]:
def newton_cotes(f, gamma1=1.7, gamma2=3.2, n_points=5,b=3.2, return_info=False):
    """ Построение квадратурной формулы Ньютона-Котеса """
    # Замена переменной
    x = np.linspace(gamma1, gamma2, n_points)
    t = b - x
    
    # Строим матрицу Вандермонда
    vander = np.vander(t, n_points, increasing=True).T
    
    # Находим моменты
    mu = np.array([moment(gamma1, gamma2, k) for k in range(len(x))])
    
    # Находим коэф в квадратурной форме,
    A = np.linalg.solve(vander, mu)
    
    if return_info is True:
        return {"mu": mu, "A": A, "S": sum(A * f(x))}
    
    return sum(A * f(x))

In [53]:
def gauss(f, gamma1=1.7, gamma2=3.2, b=3.2, n_points=10):
    """ Построение квадратурной формулы с помощью специального выбора узлов.
        Алгебраическа степень точности 2*n - 1, где n - число узлов """
    
    # Находим моменты
    moments = np.array([moment(gamma1, gamma2, k) for k in range(2*n_points)])
    
    # Решаем систему, чтобы найти коэф. узлового многочлена
    maxtix_of_moments = np.array([moments[s: s + n_points] for s in range(n_points)])
    poly_coeffs = np.linalg.solve(maxtix_of_moments, -moments[n_points:])
    
    # Строим полином, находим узлы квадратурной формы
    poly = np.poly1d([1] + list(reversed(poly_coeffs)))
    nodes = poly.roots
    
    # Из-за вычислительной погрешности, корни могут находиться вне интервала а, b 
    # выведем предупреждение, если это так
    if not ((gamma1 - b < - nodes).all() and (-nodes < 0).all()):
        print("Некоторые узлы выходят за промежуток интегрирования")
        
    # Находим коэф в квадратурной форме
    vander = np.vander(nodes, n_points, increasing=True).T
    quad_coeff = np.linalg.solve(vander, moments[:n_points])
    
    return sum(f(b - nodes) * quad_coeff)

In [54]:
gauss(f, n_points=5)

23.576656048069374

In [55]:
# Составная квадратурная формула
def quad(f, a=1.7, b=3.2, method="newton_cotes", n_parts=10, n_points=10):
    if method == "newton_cotes":
        method = newton_cotes
    if method == "gauss":
        method = gauss
    
    # Разбиваем на интервалы
    intervals = np.linspace(a, b, n_parts)
    integral = 0
    for i in range(1, len(intervals)):
        # На каждом интервале применяем заданную квадратурную форму 
        integral += method(f, intervals[i - 1], intervals[i], n_points=n_points)
        
    return integral

In [60]:
quad(f, method="gauss", n_points=5)

23.576655383704455

In [61]:
quad(f, method="newton_cotes", n_points=10)

23.57665538370445